<a href="https://colab.research.google.com/github/gbr-guimaraes/Projeto-organizador-de-estudos/blob/main/Projeto_Cronograma_de_estudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação das bibliotecas

In [69]:
!pip install -q -U google-generativeai  # Instala a biblioteca google-generativeai
!pip install PyPDF2 -q                  # Instala a biblioteca PyPDF2

#Importação das dependencias

In [70]:
from google.colab import userdata
import google.generativeai as genai
import os
import PyPDF2
import ipywidgets as widgets
from IPython.display import display

#Definição das variaveis de configuração

In [71]:
API_KEY = userdata.get('SECRET_KEY')  # Obtém a chave de API do Gemni
max_token = 1048576                   # Define o limite máximo de tokens para o modelo

#Configuração do modelo

In [72]:
genai.configure(api_key=os.environ["API_KEY"])          # Configura a API com a chave
model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Define o modelo a ser usado

#Função para extrair o texto de um PDF em partes menores

In [73]:
def extract_pdf_pages(pathname: str) -> list[str]:
    parts = [f"--- START OF PDF ${pathname} ---"]       # Inicia a lista de partes com um cabeçalho
    with open(pathname, "rb") as pdf_file:              # Abre o arquivo PDF em modo binário
        pdf_reader = PyPDF2.PdfReader(pdf_file)         # Cria um objeto leitor de PDF
        num_pages = len(pdf_reader.pages)               # Obtém o número de páginas
        for page_num in range(num_pages):               # Itera sobre cada página
            page = pdf_reader.pages[page_num]           # Obtém a página atual
            page_text = page.extract_text()             # Extrai o texto da página
            if len(parts) + len(page_text) > max_token: # Verifica se a parte excede o limite de tokens
                # Divide a entrada em partes menores
                convo.send_message(parts)
                parts = []
            parts.append(f"--- PAGE {page_num} ---")    # Adiciona um cabeçalho para a página
            parts.append(page_text)                     # Adiciona o texto da página à lista de partes
    return parts

# Função para obter a lista de cargos disponíveis no edital

In [74]:
def role_list_generate():
    convo = model.start_chat(history=[])                          # Inicia uma conversa com o modelo
    convo.send_message(extract_pdf_pages("/content/edital.pdf"))  # Envia o texto do PDF ao modelo
    # Solicita ao modelo que apresente os cargos disponíveis em formato de lista
    convo.send_message("Apresente os cargos disponiveis neste edital em formato de lista simples,apenas texto, um elemento em cada linha, sem detalhes, textos de introdução ou caracteres especiais")
    return convo.last.text                                        # Retorna a resposta do modelo

# Função para gerar um cronograma de estudos personalizado

In [83]:
def call_gemni(role: str, student_name: str, week_days: str, hours):
    convo = model.start_chat(history=[])                          # Inicia uma conversa com o modelo
    convo.send_message(extract_pdf_pages("/content/edital.pdf"))  # Envia o texto do PDF ao modelo
    # Solicita ao modelo que crie um cronograma de estudos com base nas informações fornecidas
    convo.send_message(f"A partir do arquivo edital.pdf crie um cronograma de estudos personalizado, em formato de tabela, com os conteudos gerais e especificos para o cargo de {role} para o estudante {student_name}, este aluno tem disponibilidade de estudar {hours} horas por dia, nos dias {week_days}")
    print(convo.last.text)                                        # Imprime o cronograma gerado pelo modelo

# Função para atualizar a lista de cargos disponíveis após o upload do arquivo

In [76]:
def handle_file_upload(change):
    uploaded_file = change['new']
    file_save()
    roles_list = []
    roles_list = role_list_generate().splitlines() # Obtém a lista de cargos do modelo
    roles.options = roles_list                     # Atualiza as opções do widget de seleção de cargos

# Função para processar o clique no botão "Gerar"

In [77]:
def on_button_clicked(b):
    # Obtém os dias da semana selecionados pelo usuário
    checkboxes = [monday, tuesday, wednesday, thursday, friday, saturday, sunday]
    week_list = ""
    for checkbox in checkboxes:
        if checkbox.value:
            week_list += checkbox.description + ", "
    # Chama a função para gerar o cronograma com as informações do usuário
    call_gemni(roles.value, name.value, week_list, time_avaliable.value)

# Função para salvar o arquivo PDF carregado pelo usuário

In [78]:
def file_save():
    if file_upload.data != []:
      with open("./edital.pdf", "wb") as fp:
          fp.write(file_upload.data[0])

# Criação dos widgets da interface do usuário

In [84]:
# Widget para upload do arquivo PDF
file_upload = widgets.FileUpload(accept='.pdf',multiple=False)
# Widgets para seleção dos dias da semana disponíveis para estudo
monday = widgets.Checkbox(value=False, description='Segunda-Feira', disabled=False, indent=False)
tuesday = widgets.Checkbox(value=False, description='Terça-Feira', disabled=False, indent=False)
wednesday = widgets.Checkbox(value=False, description='Quarta-Feira', disabled=False, indent=False)
thursday = widgets.Checkbox(value=False, description='Quinta-Feira', disabled=False, indent=False)
friday = widgets.Checkbox(value=False, description='Sexta-Feira', disabled=False, indent=False)
saturday = widgets.Checkbox(value=False, description='Sábado', disabled=False, indent=False)
sunday = widgets.Checkbox(value=False, description='Domingo', disabled=False, indent=False)
# Widget para definir o número de horas disponíveis por dia
time_avaliable = widgets.BoundedIntText(value=5, min=1, max=24, step=1, description='',disabled=False)
# Widget para inserir o nome do usuário
name = widgets.Text(value='', placeholder='Nome...', description='', disabled=False)
# Widget para selecionar o cargo desejado
roles = widgets.Dropdown(options=[], description='', disabled=False,)
# Botão "Gerar"
submit = widgets.ToggleButton(value=False, description='Gerar', disabled=False, button_style='', tooltip='Description', icon='check')

# Exibição dos widgets na interface e eventos

In [85]:
print("Faça o upload do seu edital:")
display(file_upload)
print("\nSeu Nome:")
display(name)
print("\nCargo desejado:")
display(roles)
print("\nDias disponiveis para estudo:")
display(monday, tuesday, wednesday, thursday, friday, saturday, sunday)
print('\nHoras disponiveis por dia:')
display(time_avaliable)
print("\n")
display(submit)
print("\n\n")

# Vincula as funções aos eventos dos widgets
file_upload.observe(handle_file_upload, names='value') # Chama a função quando um arquivo é carregado
submit.observe(on_button_clicked)                      # Chama a função quando o botão é clicado

Faça o upload do seu edital:


FileUpload(value={}, accept='.pdf', description='Upload')


Seu Nome:


Text(value='', placeholder='Nome...')


Cargo desejado:


...

Checkbox(value=False, description='Domingo', indent=False)


Horas disponiveis por dia:


BoundedIntText(value=5, max=24, min=1)

ToggleButton(value=False, description='Gerar', icon='check', tooltip='Description')




## Cronograma de Estudos Personalizado - CPNU 2024 - Analista em Tecnologia da Informação (ATI) - Gabriel

**Disponibilidade:** 5 horas por dia (Segunda, Terça, Quinta, Sexta, Sábado)

**Observações:**

* Este cronograma é um modelo e pode ser ajustado conforme as necessidades e o ritmo de aprendizagem do Gabriel.
* Recomenda-se a utilização de materiais de estudo como livros, cursos online, videoaulas e resolução de exercícios.
* É importante incluir pausas regulares para descanso e evitar a exaustão. 
* O tempo dedicado a cada eixo temático pode variar de acordo com a familiaridade do Gabriel com o conteúdo. 

**Legenda:**

* CG: Conhecimentos Gerais
* CE: Conhecimentos Específicos
* ET1: Eixo Temático 1 - Gestão Governamental e Governança Pública
* ET2: Eixo Temático 2 - Políticas Públicas
* ET3: Eixo Temático 3 - Gerência e Suporte da Tecnologia da Informação
* ET4: Eixo Temático 4 - Desenvolvimento de Software 
* ET5: Eixo Temático 5 - Apoio à Decisão, Inteligência Artificial e